In [1]:
from langchain.document_loaders import CSVLoader
 
loader = CSVLoader("dataset/summary_results.csv")
data = loader.load()
 
from langchain.text_splitter import RecursiveCharacterTextSplitter
 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4096,
    chunk_overlap  = 400,
    length_function = len,
    is_separator_regex = False
)

all_splits = text_splitter.split_documents(data)

In [12]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS "] = "%APPDATA%\gcloud\application_default_credentials.json"

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings 
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2') 

from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(data, embeddings)
vectorstore.save_local('vector_store/rag_data')

In [13]:
from langchain.llms import VertexAI

llm = VertexAI()

PermissionDenied: 403 Your application is authenticating by using local Application Default Credentials. The aiplatform.googleapis.com API requires a quota project, which is not set by default. To learn how to set your quota project, see https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds . [reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "consumer"
  value: "projects/764086051850"
}
]

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
 
chat_model = ChatOpenAI(openai_api_key = "sk-no-key-required", 
                        openai_api_base = "http://localhost:8080/v1", 
                        max_tokens=8192,
                        temperature=0.2)
 
qa_chain = RetrievalQA.from_chain_type(chat_model, 
                                       retriever=vectorstore.as_retriever(search_kwargs={"k": 1}))

print(qa_chain({"query": "請推薦一間牛肉麵"})['result'])

ImportError: Could not import openai python package. Please install it with `pip install openai`.